<a href="https://colab.research.google.com/github/nikhilnimbalkar1/Coursera_Capstone/blob/master/BattleOfTheNeighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Battle of Negiborhoods - Mumbai

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


import numpy as np # library to handle data in a vectorized manner


import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
url = "https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai"
html_data = requests.get(url).content
soup = BeautifulSoup(html_data,"html5lib")
cols = ['Area','Location','Latitude','Longitude']
mumbai_data = pd.DataFrame(columns=cols)
table_data = soup.find('table',attrs={'class':'wikitable sortable'})
for tr in table_data.findAll('tr'):
  row_data = []
  for td in tr.findAll('td'):
    row_data.append(td.text)
  mumbai_data = mumbai_data.append(dict(zip(cols,row_data)),ignore_index = True)

mumbai_data = mumbai_data.replace(r'\n','',regex=True)
mumbai_data.dropna(inplace=True)
mumbai_data[["Longitude","Latitude"]] = mumbai_data[["Longitude","Latitude"]].astype(float)
  
mumbai_data

,Area,Location,Latitude,Longitude
1,Amboli,"Andheri,Western Suburbs",19.129300,72.843400
2,"Chakala, Andheri",Western Suburbs,19.111388,72.860833
3,D.N. Nagar,"Andheri,Western Suburbs",19.124085,72.831373
4,Four Bungalows,"Andheri,Western Suburbs",19.124714,72.827210
5,Lokhandwala,"Andheri,Western Suburbs",19.130815,72.829270
6,Marol,"Andheri,Western Suburbs",19.119219,72.882743
7,Sahar,"Andheri,Western Suburbs",19.098889,72.867222
8,Seven Bungalows,"Andheri,Western Suburbs",19.129052,72.817018
9,Versova,"Andheri,Western Suburbs",19.120000,72.820000
10,Mira Road,"Mira-Bhayandar,Western Suburbs",19.284167,72.871111


In [3]:
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai City are 19.0759899, 72.8773928.


In [4]:
mumbai_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(mumbai_data['Latitude'], mumbai_data['Longitude'], mumbai_data['Area'], mumbai_data['Location']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mumbai_map)  
    
mumbai_map

In [5]:
CLIENT_ID = 'O0O3FX0PTLWU53I2CE5OQDQ1L55U53HXN14NVR5JO3SVKO4S'
CLIENT_SECRET = 'I2KCAJ21VKBENTBC2Q0KVPYWXVPTJ5M5MBTWWTD5AZKLUO3M'
VERSION = '20180605'
LIMIT = 100

In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Location', 
                  'Location Latitude', 
                  'Location Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
mumbai_venues = getNearbyVenues(names=mumbai_data['Location'],
                                   latitudes=mumbai_data['Latitude'],
                                   longitudes=mumbai_data['Longitude']
                                  )

Andheri,Western Suburbs
Western Suburbs
Andheri,Western Suburbs
Andheri,Western Suburbs
Andheri,Western Suburbs
Andheri,Western Suburbs
Andheri,Western Suburbs
Andheri,Western Suburbs
Andheri,Western Suburbs
Mira-Bhayandar,Western Suburbs
Mira-Bhayandar,Western Suburbs
Mira-Bhayandar,Western Suburbs
Bandra,Western Suburbs
Bandra,Western Suburbs
Bandra,Western Suburbs
Borivali (West),Western Suburbs
Borivali (West),Western Suburbs
Western Suburbs
Goregaon,Western Suburbs
Goregaon,Western Suburbs
Western Suburbs
Western Suburbs
Kandivali West,Western Suburbs
Kandivali West,Western Suburbs
Kandivali West,Western Suburbs
Kandivali East,Western Suburbs
Khar,Western Suburbs
Khar,Western Suburbs
Malad,Western Suburbs
Malad,Western Suburbs
Sanctacruz,Western Suburbs
Vasai,Western Suburbs
Vasai,Western Suburbs
Western Suburbs
Vile Parle,Western Suburbs
Western Suburbs
Eastern Suburbs
Ghatkopar,Eastern Suburbs
Ghatkopar,Eastern Suburbs
Ghatkopar,Eastern Suburbs
Eastern Suburbs
Kurla,Eastern Subu

In [8]:
mumbai_venues.head()

,Location,Location Latitude,Location Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Andheri,Western Suburbs",19.1293,72.8434,Cafe Arfa,19.128930,72.847140,Indian Restaurant
1,"Andheri,Western Suburbs",19.1293,72.8434,"5 Spice , Bandra",19.130421,72.847206,Chinese Restaurant
2,"Andheri,Western Suburbs",19.1293,72.8434,Subway,19.127860,72.844461,Sandwich Place
3,"Andheri,Western Suburbs",19.1293,72.8434,Cafe Coffee Day,19.127748,72.844663,Coffee Shop
4,"Andheri,Western Suburbs",19.1293,72.8434,Apple Service Centre,19.128036,72.842718,IT Services


In [9]:
mumbai_venues.groupby('Location').count()

,Location Latitude,Location Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Location,,,,,,
"Andheri,Western Suburbs",99,99,99,99,99,99
"Antop Hill,South Mumbai",6,6,6,6,6,6
"Bandra,Western Suburbs",111,111,111,111,111,111
"Borivali (West),Western Suburbs",22,22,22,22,22,22
"Byculla,South Mumbai",7,7,7,7,7,7
"Colaba,South Mumbai",2,2,2,2,2,2
Eastern Suburbs,16,16,16,16,16,16
"Fort,South Mumbai",6,6,6,6,6,6
"Ghatkopar,Eastern Suburbs",53,53,53,53,53,53


In [10]:
mumbai_venue_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(mumbai_venues['Venue Latitude'], mumbai_venues['Venue Longitude'], mumbai_venues['Location'], mumbai_venues['Venue']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
        parse_html=False).add_to(mumbai_venue_map)  
    
mumbai_venue_map

In [11]:
# one hot encoding
mumbai_onehot = pd.get_dummies(mumbai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mumbai_onehot['Location'] = mumbai_venues['Location'] 

# move neighborhood column to the first column
fixed_columns = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
mumbai_onehot = mumbai_onehot[fixed_columns]

mumbai_onehot.head()

,Location,Accessories Store,Afghan Restaurant,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Bus Station,Business Service,Café,Campground,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Auditorium,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,History Museum,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Moving Target,Mughlai Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Other Great Outdoors,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Souvenir Shop,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Tea Room,Tennis Court,Tennis Stadium,Tex-Mex Restaurant,Theater,Tourist Information Center,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Andheri,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Andheri,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Andheri,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Andheri,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Andheri,Western Suburbs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [12]:
mumbai_grouped = mumbai_onehot.groupby('Location').mean().reset_index()
mumbai_grouped

,Location,Accessories Store,Afghan Restaurant,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Bus Station,Business Service,Café,Campground,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Auditorium,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,History Museum,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Moving Target,Mughlai Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Other Great Outdoors,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Souvenir Shop,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Tea Room,Tennis Court,Tennis Stadium,Tex-Mex Restaurant,Theater,Tourist Information Center,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Andheri,Western Suburbs",0.020202,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010101,0.000000,0.000000,0.020202,0.000000,0.040404,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.010101,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.010101,0.000000,0.050505,0.000000,0.000000,0.000000,0.040404,0.020202,0.000000,0.020202,0.040404,0.000000,0.000000,0.010101,0.010101,0.000000,0.010101,0.0000,0.000000,0.000000,0.000000,0.020202,0.000000,0.000000,0.010101,0.020202,0.000000,0.000000,0.010101,0.000000,0.000000,0.000000,0.010101,0.000000,0.010101,0.010101,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020202,0.020202,0.000000,0.000000,0.000000,0.000000,0.010101,0.000000,0.010101,0.040404,0.000000,0.101010,0.000000,0.000000,0.010101,0.000000,0.000000,0.030303,0.000000,0.000000,0.00,0.000000,0.010101,0.010101,0.000000,0.020202,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.010101,0.000000,0.000000,0.000000,0.000000,0.010101,0.000000,0.000000,0.010101,0.000000,0.000000,0.030303,0.040404,0.000000,0.000000,0.000000,0.00,0.040404,0.010101,0.020202,0.000000,0.010101,0.000000,0.000000,0.010101,0.010101,0.000000,0.020202,0.000000,0.000000,0.000000,0.010101,0.020202,0.000000,0.010101,0.000000,0.000000,0.000000,0.0000,0.010101,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020202,0.000000,0.000000,0.000000,0.020202,

In [13]:
num_top_venues = 5

for hood in mumbai_grouped['Location']:
    print("----"+hood+"----")
    temp = mumbai_grouped[mumbai_grouped['Location'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Andheri,Western Suburbs----
                venue  freq
0   Indian Restaurant  0.10
1                Café  0.05
2         Pizza Place  0.04
3  Chinese Restaurant  0.04
4                 Bar  0.04


----Antop Hill,South Mumbai----
                  venue  freq
0           Pizza Place  0.17
1  Fast Food Restaurant  0.17
2          Dessert Shop  0.17
3                Garden  0.17
4                  Café  0.17


----Bandra,Western Suburbs----
                venue  freq
0   Indian Restaurant  0.07
1                Café  0.06
2              Bakery  0.06
3      Ice Cream Shop  0.05
4  Italian Restaurant  0.05


----Borivali (West),Western Suburbs----
                venue  freq
0  Chinese Restaurant  0.14
1   Indian Restaurant  0.09
2              Resort  0.09
3      Ice Cream Shop  0.05
4                 Bar  0.05


----Byculla,South Mumbai----
               venue  freq
0  Indian Restaurant  0.43
1              Diner  0.14
2       Cupcake Shop  0.14
3        Art Gallery  0.14
4      Gr

In [14]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [15]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Location']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Location'] = mumbai_grouped['Location']

for ind in np.arange(mumbai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mumbai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Andheri,Western Suburbs",Indian Restaurant,Café,Chinese Restaurant,Ice Cream Shop,Pub,Pizza Place,Bar,Coffee Shop,Juice Bar,Pharmacy
1,"Antop Hill,South Mumbai",Fast Food Restaurant,Garden,Pizza Place,Liquor Store,Dessert Shop,Café,Farmers Market,Falafel Restaurant,Event Space,Electronics Store
2,"Bandra,Western Suburbs",Indian Restaurant,Bakery,Café,Ice Cream Shop,Fast Food Restaurant,Italian Restaurant,Dessert Shop,Bar,Cupcake Shop,Scenic Lookout
3,"Borivali (West),Western Suburbs",Chinese Restaurant,Indian Restaurant,Resort,Liquor Store,Dessert Shop,Bar,Bakery,Fast Food Restaurant,Ice Cream Shop,Tea Room
4,"Byculla,South Mumbai",Indian Restaurant,Cupcake Shop,Grocery Store,Art Gallery,Diner,Yoga Studio,Donut Shop,Field,Fast Food Restaurant,Farmers Market


In [16]:
# set number of clusters
kclusters = 5

mumbai_grouped_clustering = mumbai_grouped.drop('Location', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 0, 4, 1, 4, 1, 3], dtype=int32)

In [17]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

mumbai_merged = mumbai_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
mumbai_merged = mumbai_merged.join(neighborhoods_venues_sorted.set_index('Location'), on='Location',lsuffix="l")

mumbai_merged.head() # check the last columns!

,Area,Location,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Amboli,"Andheri,Western Suburbs",19.129300,72.843400,1.0,Indian Restaurant,Café,Chinese Restaurant,Ice Cream Shop,Pub,Pizza Place,Bar,Coffee Shop,Juice Bar,Pharmacy
2,"Chakala, Andheri",Western Suburbs,19.111388,72.860833,1.0,Indian Restaurant,Bar,Hotel,Café,Fast Food Restaurant,Dessert Shop,Restaurant,Spa,Pizza Place,Ice Cream Shop
3,D.N. Nagar,"Andheri,Western Suburbs",19.124085,72.831373,1.0,Indian Restaurant,Café,Chinese Restaurant,Ice Cream Shop,Pub,Pizza Place,Bar,Coffee Shop,Juice Bar,Pharmacy
4,Four Bungalows,"Andheri,Western Suburbs",19.124714,72.827210,1.0,Indian Restaurant,Café,Chinese Restaurant,Ice Cream Shop,Pub,Pizza Place,Bar,Coffee Shop,Juice Bar,Pharmacy
5,Lokhandwala,"Andheri,Western Suburbs",19.130815,72.829270,1.0,Indian Restaurant,Café,Chinese Restaurant,Ice Cream Shop,Pub,Pizza Place,Bar,Coffee Shop,Juice Bar,Pharmacy


In [18]:
mumbai_merged.dropna(inplace = True)

In [19]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_merged['Latitude'], mumbai_merged['Longitude'], mumbai_merged['Location'], mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [23]:
mumbai_merged.loc[mumbai_merged["Cluster Labels"] == 1].head()

,Area,Location,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Amboli,"Andheri,Western Suburbs",19.129300,72.843400,1.0,Indian Restaurant,Café,Chinese Restaurant,Ice Cream Shop,Pub,Pizza Place,Bar,Coffee Shop,Juice Bar,Pharmacy
2,"Chakala, Andheri",Western Suburbs,19.111388,72.860833,1.0,Indian Restaurant,Bar,Hotel,Café,Fast Food Restaurant,Dessert Shop,Restaurant,Spa,Pizza Place,Ice Cream Shop
3,D.N. Nagar,"Andheri,Western Suburbs",19.124085,72.831373,1.0,Indian Restaurant,Café,Chinese Restaurant,Ice Cream Shop,Pub,Pizza Place,Bar,Coffee Shop,Juice Bar,Pharmacy
4,Four Bungalows,"Andheri,Western Suburbs",19.124714,72.827210,1.0,Indian Restaurant,Café,Chinese Restaurant,Ice Cream Shop,Pub,Pizza Place,Bar,Coffee Shop,Juice Bar,Pharmacy
5,Lokhandwala,"Andheri,Western Suburbs",19.130815,72.829270,1.0,Indian Restaurant,Café,Chinese Restaurant,Ice Cream Shop,Pub,Pizza Place,Bar,Coffee Shop,Juice Bar,Pharmacy
